In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
from PIL import Image
import numpy as np
import os

def preprocess_image(image_path):
    images = Image.open(image_path)
    images3D = images.repeat(3, -1)  
    images3D_resized = tf.image.resize(images3D, [64, 64])
    img_array = np.array(images3D_resized) / 255.0  # 归一化像素值
    return img_array

In [4]:

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 使用 datagen.flow() 生成增强的数据

In [19]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [6]:
from keras import layers, models
from keras.regularizers import l1_l2

x = layers.GlobalAveragePooling2D()(base_model.output)

x = layers.Dense(units=128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(x)

output = layers.Dense(units=5, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=output)


In [139]:
output = layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001))(base_model.output)
output = layers.Dense(5, activation='softmax')(output)  # 根据您的5个分类设置输出层

model = models.Model(inputs=base_model.input, outputs=output)

In [8]:
loss_function = tf.keras.losses.categorical_crossentropy

In [9]:
metrics = [tf.keras.metrics.CategoricalAccuracy()]

In [31]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# 设置优化器，使用初始学习率
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

In [32]:
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
batch_size = 32
train_data_directory = 'D:/DISCOVER-main/IVF/IMAGES/train'

# 创建数据生成器并指定从训练集中划分 20%作为验证集
train_datagen = ImageDataGenerator(validation_split=0.20)

# 生成训练集数据
train_generator = train_datagen.flow_from_directory(
    train_data_directory,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# 生成验证集数据
validation_generator = train_datagen.flow_from_directory(
    train_data_directory,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 1203 images belonging to 5 classes.
Found 298 images belonging to 5 classes.


In [13]:
print("Training data shape:", train_generator[0][0].shape)
print("Training labels shape:", train_generator[0][1].shape)

Training data shape: (32, 64, 64, 3)
Training labels shape: (32, 5)


In [29]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[reduce_lr]
)


Epoch 1/20
38/38 [==============================] - 4s 44ms/step - loss: 19.9164 - accuracy: 0.8421 - val_loss: 19.7781 - val_accuracy: 0.2349 - lr: 0.0100
Epoch 2/20
38/38 [==============================] - 1s 31ms/step - loss: 16.3086 - accuracy: 0.7880 - val_loss: 16.6460 - val_accuracy: 0.3725 - lr: 0.0100
Epoch 3/20
38/38 [==============================] - 1s 31ms/step - loss: 12.6265 - accuracy: 0.9019 - val_loss: 13.0174 - val_accuracy: 0.4463 - lr: 0.0100
Epoch 4/20
38/38 [==============================] - 1s 32ms/step - loss: 9.5943 - accuracy: 0.9534 - val_loss: 8.7505 - val_accuracy: 0.7349 - lr: 0.0100
Epoch 5/20
38/38 [==============================] - 1s 32ms/step - loss: 6.9736 - accuracy: 0.9884 - val_loss: 6.7486 - val_accuracy: 0.6544 - lr: 0.0100
Epoch 6/20
38/38 [==============================] - 1s 31ms/step - loss: 4.8878 - accuracy: 0.9909 - val_loss: 4.5324 - val_accuracy: 0.7819 - lr: 0.0100
Epoch 7/20
38/38 [==============================] - 1s 31ms/step - los

In [30]:
test_datagen = ImageDataGenerator(
    # 测试集的数据增强参数（如果有）
)
test_generator = test_datagen.flow_from_directory(
    'D:/DISCOVER-main/IVF/IMAGES/test1',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'  # 对于多分类问题
)
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Found 377 images belonging to 5 classes.
12/12 [==============================] - 0s 29ms/step - loss: 2.4806 - accuracy: 0.1379
Test loss: 2.480602741241455
Test accuracy: 0.13793103396892548


In [119]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    'D:/DISCOVER-main/IVF/IMAGES/test1',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Found 377 images belonging to 5 classes.
12/12 [==============================] - 1s 24ms/step
Test loss: 3.0832865238189697
Test accuracy: 0.23342175781726837


In [26]:
class_names = ['CAL27', 'hepG2', 'sacc_83', 'sacc_LM', 'A431']

# 生成混淆矩阵
cm = confusion_matrix(true_labels, predicted_labels, labels=[0, 1, 2, 3, 4]) 

plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_names))

# 将类别名称设置为 class_names 的顺序
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

# 在混淆矩阵上标注数字
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()


NameError: name 'true_labels' is not defined